# S&P 80 Index 

## Library Imports
Importing the important open-source software libraries 

In [2]:
import numpy as np
import pandas as pd
import math
import requests
import xlsxwriter
import random

# Importing Our List of Stocks

We are importing our list of stocks from <span style='background:#D3D3D3'> "STOCKS_DATA.csv"   </span>

In [5]:
filename = "STOCKS_DATA.csv"
total_number_of_stocks = sum(1 for line in open(filename)) - 1
limit = 80
skip = sorted(random.sample(range(1, total_number_of_stocks + 1), total_number_of_stocks - limit))
df = pd.read_csv(filename, skiprows = skip)
# print(df.to_string())
# print(df)

# Acquiring an API Token

We are importing our TwelveData API token. This is the data provider that we are using in our project.

In [3]:
from onepiece import API_KEY

# Making Our First API Call

Now it's time to structure our API calls to TwelveData

The Data we need are :
- Price of stock
- Volume of stock
- Excahange name

In [4]:
symbol = 'ZTS'
api_url = f'https://api.twelvedata.com/time_series?start_date=2023-06-30&outputsize=1&symbol={symbol}&interval=1day&apikey={API_KEY}'
data = requests.get(api_url).json()
print(data)

{'meta': {'symbol': 'ZTS', 'interval': '1day', 'currency': 'USD', 'exchange_timezone': 'America/New_York', 'exchange': 'NYSE', 'mic_code': 'XNYS', 'type': 'Common Stock'}, 'values': [{'datetime': '2023-06-30', 'open': '173.28999', 'high': '174.67999', 'low': '171.67000', 'close': '172.21001', 'volume': '2061100'}], 'status': 'ok'}


In [8]:
# print(df)

### Storing all Stocks name in one particular List

In [9]:
stocks_name = []
for x in df['Ticker']:
    stocks_name.append(x)
    
# print(stocks_name)
# print(len(stocks_name) % 8)

### Storing every 8 stocks in one List 

(since we are only allowed 8 API call per minute from TwelveData)

In [10]:
st = []
for i in range(0, len(stocks_name), 8):
    new_st = stocks_name[i : i + 8]
    st.append(new_st)
    # print(new_st)
    
# print(st)

# Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we are pulling data for all the accessible stocks available in our List and storing particular imformation in their particular list.

In [11]:
ticker_name = []
price = []
volume_val = []
shares_to_buy = []
exchange = []
import time 
for i in range(0, len(st)):
    a = st[i]
    for ticker in a:
        batch_api_url = f'https://api.twelvedata.com/time_series?start_date=2023-06-30&outputsize=1&symbol={ticker}&interval=1day&apikey={API_KEY}'
        data = requests.get(batch_api_url).json()
        if data['status'] == 'ok':
            ticker_name.append(data['meta']['symbol'])
            price.append(float(data['values'][0]['open']))
            volume_val.append(float(data['values'][0]['volume']))
            shares_to_buy.append('N/A')
            exchange.append(data['meta']['exchange'])
        # print(data)
    time.sleep(80)


# Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price, volume, exchange and ticker's name to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [12]:
data_set = {
    'Ticker' : ticker_name,
    'Price' : price,
    'Shares to Buy' : shares_to_buy,
    'Exchange' : exchange,
    'Volume' : volume_val
}
final_df = pd.DataFrame(data_set)
print(final_df)

   Ticker      Price Shares to Buy Exchange      Volume
0     AAL   17.72000           N/A   NASDAQ  18597900.0
1    ADSK  207.42000           N/A   NASDAQ   2094800.0
2     ALK   52.54000           N/A     NYSE   1555200.0
3    AMCR    9.91000           N/A     NYSE   6019900.0
4     AMP  334.66000           N/A     NYSE    499400.0
..    ...        ...           ...      ...         ...
72    VNO   17.49000           N/A     NYSE  11546800.0
73    WAT  264.26999           N/A     NYSE    882800.0
74    WST  380.10999           N/A     NYSE    353400.0
75    XYL  112.23000           N/A     NYSE   1278100.0
76    ZBH  146.23000           N/A     NYSE   1281900.0

[77 rows x 5 columns]


# Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [13]:
portfolio_size = input("Enter the value of your portfolio: ")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again")
    portfolio_size = input('Enter the value of your portfolio: ')

In [15]:
position_size = float(portfolio_size) / len(final_df.index)
for i in range(0, len(final_df['Ticker'])):
    final_df.loc[i, 'Shares to Buy'] = math.floor(position_size / final_df['Price'][i])
    
print(final_df.to_string())

   Ticker      Price Shares to Buy Exchange       Volume
0     AAL   17.72000         29316   NASDAQ   18597900.0
1    ADSK  207.42000          2504   NASDAQ    2094800.0
2     ALK   52.54000          9887     NYSE    1555200.0
3    AMCR    9.91000         52419     NYSE    6019900.0
4     AMP  334.66000          1552     NYSE     499400.0
5     AMT  194.27000          2674     NYSE    3243100.0
6     BIO  377.57999          1375     NYSE     156300.0
7      BK   44.64000         11637     NYSE    3402400.0
8     BSX   53.79000          9657     NYSE    7238600.0
9    CARR   49.64000         10464     NYSE    4305900.0
10   CBOE  137.14000          3787     CBOE     525800.0
11    CCL   17.80000         29184     NYSE  114966800.0
12     CF   70.73000          7344     NYSE    2095300.0
13     CI  277.64001          1871     NYSE    1525000.0
14    CMI  243.81000          2130     NYSE     849900.0
15    CVX  157.46001          3299     NYSE    6346600.0
16    DIS   89.18000          5

# Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

## Initializing our XlsxWriter Object

In [101]:
writer = pd.ExcelWriter('Trades.xlsx', engine = 'xlsxwriter')
final_df.to_excel(writer, sheet_name = 'Trades', index = False, header = False)

# Creating the Formats We'll Need For Our <span style='background:#D3D3D3'> .xlsx  </span> File

Formats include colors, fonts, and also symbols like <span style='background:#D3D3D3'> %  </span>  and <span style='background:#D3D3D3'> $  </span>. We'll need four main formats for our Excel document:

- String format for tickers

- $XX.XX format for stock prices

- String format for exchange

- XXX,XXX format for volume



In [102]:
background_color = 'white'
font_color = 'black'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color' : background_color,
        'border' : 1
    })


dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        })


integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        })

# Applying the Formats to the Columns of Our <span style='background:#D3D3D3'> .xlsx  </span>  File

We can use the <span style='background:#D3D3D3'> set_column  </span> method applied to the <span style='background:#D3D3D3'> writer.sheets['Trades']  </span> object to apply formats to specific columns of our spreadsheets.

Here's an example:

```
writer.sheets['Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [103]:
column_formats = {
    'A' : ['Ticker', string_format],
    'B' : ['Price', dollar_format],
    'C' : ['Shares to Buy', integer_format],
    'D' : ['Exchange', string_format],
    'E' : ['Volume', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Trades'].write(f'{column}1', column_formats[column][0], string_format)

# Saving Our Excel Output

Saving our Excel file in our device.

In [104]:
writer.close()